In [18]:
# delete all rules created N months ago
from datetime import datetime, timedelta
from rucio.client import Client


In [58]:
days=0

In [20]:
rucio=Client()
me=rucio.whoami()
account=me['account']
print(f"using Rucio account: {account}")

using Rucio account: belforte


In [21]:
# get rules for this account
ruleGen = rucio.list_replication_rules({'account': account})
rules = list(ruleGen)
print(f"{len(rules)} rules exist for account: {account}")

64 rules exist for account: belforte


In [64]:
def getRuleQuota(id):
    """ return quota needed by this rule in MB """
    size = 0
    ruleG = rucio.list_replication_rules({'id':id})
    try:
        rule=next(ruleG)
        replicaG = rucio.list_replicas([{'scope': rule['scope'], 'name': rule['name']}])
        for rep in replicaG:
            size += rep['bytes']
    except StopIteration:
        pass

    return size/1e6

In [65]:
id='a256abc2f93f40eba5b3b0107d686956'
quota = getRuleQuota(id)
print(quota)

298.306411


In [69]:
now = datetime.now()
totalSize = 0
totalRules = 0
for rule in rules:
    created=rule['created_at']
    size = getRuleQuota(rule['id'])
    isOld = (now - created) > timedelta(days=days)
    if isOld:
        totalRules +=1
        totalSize += size
        print(f"will delete rule {rule['id']} created on {created} - {size:10.3} MB")
        #rr=rucio.delete_replication_rule(rule_id=rule['id'], purge_replicas=True)
        #if rr:
        #    print(f"deleted rule {rule['id']} created on {created}")
        #else:
        #    print(f"ERROR deletion of rule {rule['ruleId']} failed")
              
print(f"TOTAL: {totalRules} rules - {totalSize:.0f} MB = {totalSize/1e3:0.3f} GB")

will delete rule 998abb1f29b84cd5a984b531ba26f962 created on 2023-08-07 16:05:28 -      0.562 MB
will delete rule 50ee3109481f49e8928a29ab675eaf67 created on 2023-08-07 09:15:47 -      0.562 MB
will delete rule e137d7bd2d90455ca7dc438820160f34 created on 2023-08-07 09:06:06 -      0.562 MB
will delete rule 50c656c243824b88b80d4f6e646e8743 created on 2023-08-04 17:21:38 -       1.12 MB
will delete rule 3543cf8372d44ad490c4e59d5d06d6c7 created on 2023-08-04 19:20:03 -       1.12 MB
will delete rule 5b35c397491d4f7b892bc424c554cc52 created on 2023-08-06 17:15:21 -      0.562 MB
will delete rule f8e6ca5e14c1458d9536dde6e8056aa2 created on 2023-08-06 17:43:45 -      0.562 MB
will delete rule 4a3323dd1ebd43ecb358a76969a113b9 created on 2023-08-06 21:23:43 -      0.562 MB
will delete rule cbc0d24de64349dba83ef317f3faf02b created on 2023-08-06 17:27:07 -      0.562 MB
will delete rule 5ef399213c664ac2957dc9f6cdea5011 created on 2023-08-06 16:47:04 -       1.12 MB
will delete rule 5f0a7a12b4be4